In [1]:
# Let's start learning
import torch
from torch_geometric.datasets import Planetoid
from cora_loader import CitationNetwork
from torch.utils.data import DataLoader

from torch_geometric_node2vec import Node2Vec
import time

# from models import GCN,BiGCN,TriGCN
# TODO autoencoders

In [2]:
'''
    root (string): Root directory where the dataset should be saved.
    name (string): The name of the dataset (:obj:`"Cora"`,
                                            :obj:`"CiteSeer"`,
                                            :obj:`"PubMed"`).
 
'''
#cora = Planetoid(root='/tmp/Cora',name='Cora')

'\n    root (string): Root directory where the dataset should be saved.\n    name (string): The name of the dataset (:obj:`"Cora"`,\n                                            :obj:`"CiteSeer"`,\n                                            :obj:`"PubMed"`).\n \n'

In [3]:
coraDirected = CitationNetwork('/tmp/coraDirected','cora',directed=True)
coraReversed = CitationNetwork('/tmp/coraReversed','cora',directed=True,reverse=True)
coraUndirected = CitationNetwork('/tmp/coraUndirected','cora',directed=False)

In [4]:
citeseer = CitationNetwork('/tmp/citeseer','citeseer',directed=False)
citeseerDirected = CitationNetwork('/tmp/citeseerDirected','citeseer',directed=True)
citeseerReversed = CitationNetwork('/tmp/citeseerReversed','citeseer',directed=True,reverse=True)

In [5]:
PubMed = CitationNetwork('/tmp/PubMed','PubMed',directed=False)
PubMedDirected = CitationNetwork('/tmp/PubMedDirected','PubMed',directed=True)
PubMedReversed = CitationNetwork('/tmp/PubMedReversed','PubMed',directed=True,reverse=True)

In [50]:
tri,tris = eval_multiple_tri(coraUndirected,16,8,8,TriGCN,runs=100)
print('Accuracy: {:.4f}'.format(tri))

Elaplsed 00:00:37
Accuracy: 0.7868


In [21]:
undirectedGCN,undirectedGCNs = eval_multiple_bi(coraUndirected,16,16,BiGCN,runs=100)
print('Accuracy: {:.4f}'.format(undirectedGCN))

Elaplsed 00:00:29
Accuracy: 0.7803


In [25]:
biGrahpSAGE_acc,biGrahpSAGE_accs = eval_multiple_bi(coraUndirected,16,16,BiGraphSAGE,runs=100)
print('Accuracy: {:.4f}'.format(biGrahpSAGE_acc))

Elaplsed 00:00:28
Accuracy: 0.7752


In [26]:
undirectedGCN,undirectedGCNs = eval_multiple(coraUndirected,32,GCN,runs=100)
print('Accuracy: {:.4f}'.format(undirectedGCN))
undirectedGraphSAGE,undirectedGraphSAGEs = eval_multiple(coraUndirected,32,GraphSAGE,runs=100)
print('Accuracy: {:.4f}'.format(undirectedGraphSAGE))

Elaplsed 00:00:21
Accuracy: 0.7930
Elaplsed 00:00:18
Accuracy: 0.7738


Elaplsed 00:00:24
Accuracy: 0.7893
Elaplsed 00:00:39
Accuracy: 0.7341
Elaplsed 00:00:17
Accuracy: 0.7724
Elaplsed 00:00:36
Accuracy: 0.7447


In [21]:
directedGCN,directedGCNs = eval_multiple(coraDirected,16,GCN,runs=100)
print('Accuracy: {:.4f}'.format(directedGCN))
directedGraphSAGE,directedGraphSAGEs = eval_multiple(coraDirected,16,GraphSAGE,runs=100)
print('Accuracy: {:.4f}'.format(directedGraphSAGE))

Elaplsed 00:00:19
Accuracy: 0.6174
Elaplsed 00:00:17
Accuracy: 0.6821


In [64]:
biGCN_acc = eval_multiple_bi(coraUndirected,64,64,BiGCN)
print('Accuracy: {:.4f}'.format(biGCN_acc))

Elaplsed 00:02:36
Accuracy: 0.7941


In [65]:
biGrahpSAGE_acc = eval_multiple_bi(coraUndirected,64,64,BiGraphSAGE)
print('Accuracy: {:.4f}'.format(biGrahpSAGE_acc))

Elaplsed 00:02:25
Accuracy: 0.7582


In [72]:
undirectedGCN = eval_multiple(coraUndirected,128,GCN)
print('Accuracy: {:.4f}'.format(undirectedGCN))
undirectedGraphSAGE = eval_multiple(coraUndirected,128,GraphSAGE)
print('Accuracy: {:.4f}'.format(undirectedGraphSAGE))

Elaplsed 00:01:21
Accuracy: 0.7896
Elaplsed 00:01:14
Accuracy: 0.7946


In [74]:
reversedGCN = eval_multiple(coraReversed,128,GCN)
print('Accuracy: {:.4f}'.format(reversedGCN))
reversedGraphSAGE = eval_multiple(coraReversed,128,GraphSAGE)
print('Accuracy: {:.4f}'.format(reversedGraphSAGE))

Elaplsed 00:01:22
Accuracy: 0.6359
Elaplsed 00:01:12
Accuracy: 0.7553


In [1]:
import torch.nn.functional as F

In [5]:
class FeatureNNClassifier(torch.nn.Module):
    def __init__(self, n_in, n_h, n_out):
        super(FeatureNNClassifier, self).__init__()
        self.linear1 = torch.nn.Linear(n_in, n_h)
        self.linear2 = torch.nn.Linear(n_h, n_out)

    def forward(self, x):
        x = self.linear1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = F.log_softmax(x,dim=1)
        return x


In [9]:
# Define a baseline
def eval_baseline_run(dataset,data,n_h=128,n_out=7):
    n_in = dataset.num_features
#     n_out = dataset.num_classes
    baseline = FeatureNNClassifier(n_in,n_h,n_out)
    device = torch.device('cpu')
    data = dataset[0].x.to(device)
    optimizer = torch.optim.Adam(baseline.parameters(),lr=0.01,weight_decay=5e-4)
    baseline.train() # to enter training phase
    for epoch in range(200):
        optimizer.zero_grad() # saw this a lot in the beginning, maybe resetting gradients (not to accumulate)
        out = baseline(data) # this calls the forward method apparently
        loss = F.nll_loss(out[dataset[0].train_mask],dataset[0].y[dataset[0].train_mask]) # nice indexing, easy and short
        loss.backward() # magic: real back propagation step, takes care of the gradients and stuff
        optimizer.step() # maybe updates the params to be optimized
    baseline.eval() # enter eval phase
    _,pred = baseline(data).max(dim=1) # take prediction out of softmax
    correct = float(pred[dataset[0].test_mask].eq(dataset[0].y[dataset[0].test_mask]).sum().item())
    acc = correct / dataset[0].test_mask.sum().item()
    return acc
def eval_baseline(dataset,n_out,n_h=128,runs=100):
    accs = []
    for i in range(runs):
        accs.append(eval_baseline_run(dataset,n_h,n_out))
    return sum(accs)/len(accs),accs

In [23]:
baselineAcc,baselineAccs = eval_baseline(coraDirected,n_h=16,n_out=coraDirected.num_classes)
print('Accuracy: {:.4f}'.format(baselineAcc))

Accuracy: 0.5581


In [41]:
baselineAcc,baselineAccs = eval_baseline(citeseerDirected,n_h=16,n_out=citeseerDirected.num_classes)
print('Accuracy: {:.4f}'.format(baselineAcc))

Accuracy: 0.6123


In [10]:
baselineAcc,baselineAccs = eval_baseline(PubMed,n_h=16,n_out=PubMed.num_classes)
print('Accuracy: {:.4f}'.format(baselineAcc))

Accuracy: 0.6127


In [11]:


def train_n2v(n2v,data,loader,optimizer,device):
    n2v.train()
    total_loss = 0
    for subset in loader:
        optimizer.zero_grad()
        loss = n2v.loss(data.edge_index, subset.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


def test_n2v(n2v,data,device):
    n2v.eval()
    z = n2v(torch.arange(data.num_nodes, device=device))
    x = data.x
    z = torch.cat((z,x),dim=1)
    
#     acc = eval_baseline_run(data,z,n_h=16)
    acc = n2v.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask], max_iter=150)
    return acc

def run_n2v(data,embedding_dim=128,p=1,q=1):
    loader = DataLoader(torch.arange(data[0].num_nodes), batch_size=128, shuffle=True)
    device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
    n2v = Node2Vec(num_nodes=data[0].num_nodes,
              embedding_dim=embedding_dim,
              walk_length=80,
              context_size=10,
              walks_per_node=10,
              p=p,q=q)
    n2v = n2v.to(device)
    data =  data[0].to(device)
    optimizer = torch.optim.Adam(n2v.parameters(), lr=0.01)

    for epoch in range(1, 100):
        loss = train_n2v(n2v,data,loader,optimizer,device)
        if epoch % 10 == 0:
            print('Epoch: {:02d}, Loss: {:.4f}'.format(epoch, loss))
    acc = test_n2v(n2v,data,device)
    return acc,n2v

In [8]:
n2v_acc,n2v_model = run_n2v(coraUndirected)
print('Accuracy: {:.4f}'.format(n2v_acc))

Epoch: 10, Loss: 1.2064
Epoch: 20, Loss: 0.8939
Epoch: 30, Loss: 0.8566
Epoch: 40, Loss: 0.8463
Epoch: 50, Loss: 0.8422
Epoch: 60, Loss: 0.8395
Epoch: 70, Loss: 0.8390
Epoch: 80, Loss: 0.8383
Epoch: 90, Loss: 0.8379
Accuracy: 0.7670


In [8]:
n2v_acc,n2v_model = run_n2v(citeseer)
print('Accuracy: {:.4f}'.format(n2v_acc))

Epoch: 10, Loss: 1.2281
Epoch: 20, Loss: 0.8759
Epoch: 30, Loss: 0.8157
Epoch: 40, Loss: 0.7921
Epoch: 50, Loss: 0.7794
Epoch: 60, Loss: 0.7714
Epoch: 70, Loss: 0.7664
Epoch: 80, Loss: 0.7629
Epoch: 90, Loss: 0.7603
Accuracy: 0.6770


In [14]:
n2vDirected_acc,n2vDirected_model = run_n2v(PubMed)
print('Accuracy: {:.4f}'.format(n2vDirected_acc))

Epoch: 10, Loss: 0.8200
Epoch: 20, Loss: 0.8185
Epoch: 30, Loss: 0.8190
Epoch: 40, Loss: 0.8189
Epoch: 50, Loss: 0.8187
Epoch: 60, Loss: 0.8188
Epoch: 70, Loss: 0.8187
Epoch: 80, Loss: 0.8185
Epoch: 90, Loss: 0.8185
Accuracy: 0.7060


In [9]:
n2vDirected_acc,n2vDirected_model = run_n2v(coraDirected)
print('Accuracy: {:.4f}'.format(n2vDirected_acc))

Epoch: 10, Loss: 1.0266
Epoch: 20, Loss: 0.6763
Epoch: 30, Loss: 0.5729
Epoch: 40, Loss: 0.5402
Epoch: 50, Loss: 0.5274
Epoch: 60, Loss: 0.5116
Epoch: 70, Loss: 0.5037
Epoch: 80, Loss: 0.4917
Epoch: 90, Loss: 0.4880
Accuracy: 0.4770


In [8]:
n2v_acc,n2v_model = run_n2v(PubMed)
print('Accuracy: {:.4f}'.format(n2v_acc))

Epoch: 10, Loss: 0.8204
Epoch: 20, Loss: 0.8185
Epoch: 30, Loss: 0.8191
Epoch: 40, Loss: 0.8191
Epoch: 50, Loss: 0.8189
Epoch: 60, Loss: 0.8187
Epoch: 70, Loss: 0.8188
Epoch: 80, Loss: 0.8187
Epoch: 90, Loss: 0.8184
Accuracy: 0.6990


In [9]:
n2v_acc,n2v_model = run_n2v(PubMedDirected)
print('Accuracy: {:.4f}'.format(n2v_acc))

Epoch: 10, Loss: 0.3298
Epoch: 20, Loss: 0.3147
Epoch: 30, Loss: 0.3079
Epoch: 40, Loss: 0.3024
Epoch: 50, Loss: 0.2994
Epoch: 60, Loss: 0.2965
Epoch: 70, Loss: 0.2945
Epoch: 80, Loss: 0.2916
Epoch: 90, Loss: 0.2890
Accuracy: 0.3740


In [10]:
n2v_acc,n2v_model = run_n2v(PubMedReversed)
print('Accuracy: {:.4f}'.format(n2v_acc))

RuntimeError: merge_sort: failed to synchronize: an illegal memory access was encountered

In [11]:
n2v_acc,n2v_model = run_n2v(coraReversed)
print('Accuracy: {:.4f}'.format(n2v_acc))

RuntimeError: merge_sort: failed to synchronize: an illegal memory access was encountered

In [8]:
n2v_acc,n2v_model = run_n2v(citeseerDirected)
print('Accuracy: {:.4f}'.format(n2v_acc))

RuntimeError: merge_sort: failed to synchronize: an illegal memory access was encountered